In [1]:
import glob
import os

import re

import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (
    Input,
    Dense,
    Flatten,
    Reshape,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Concatenate,
    Dense,
)
from tensorflow.keras.models import Model
from keras.utils import Sequence
import typer
import scipy

2023-10-14 21:44:13.925478: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 21:44:13.925511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 21:44:13.926422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-14 21:44:14.006460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_number_from_string(s):
    s = s.split("/")[-1]
    match = re.search(r"\d+", s)
    if match:
        return int(match.group())
    else:
        return 0  # If no number is found, return 0


In [3]:
DATA_DIR = "/home/fer/Escritorio/futstatistics/datasets/dataset7_homography"
BATCH_SIZE = 8

images_1 = glob.glob(os.path.join(DATA_DIR, "test") + "/*.jpg")
images_2 = glob.glob(os.path.join(DATA_DIR, "train_val") + "/*.jpg")

matrix_1 = glob.glob(os.path.join(DATA_DIR, "test") + "/*.homographyMatrix")
matrix_2 = glob.glob(os.path.join(DATA_DIR, "train_val") + "/*.homographyMatrix")

mask_1 = glob.glob(os.path.join(DATA_DIR, "test") + "/*.mat")
mask_2 = glob.glob(os.path.join(DATA_DIR, "train_val") + "/*.mat")


images = sorted(images_1 + images_2, key=get_number_from_string)
matrices = sorted(matrix_1 + matrix_2, key=get_number_from_string)
masks = sorted(mask_1 + mask_2, key=get_number_from_string)

inputs = [(image, mask) for image, mask in zip(images, masks)]

In [4]:
train_input, val_input, train_matrices, val_matrices = train_test_split(
    inputs, matrices, test_size=0.35
)


In [44]:

class HomographyDataset(Sequence):
    def __init__(self, input_paths, labels_list, double_input, image_size, batch_size):
        self.input_paths = input_paths
        self.labels_list = labels_list
        self.batch_size = batch_size
        self.image_size = image_size
        self.double_input = double_input
        self.indices = np.arange(len(labels_list))

    def __len__(self):
        return int(np.ceil(len(self.input_paths) / self.batch_size))

    def _read_homography(self, file_path: str):
        homography_matrix = []

        with open(file_path, "r") as file:
            for line in file:
                elements = line.strip().split()

                row = [float(element) for element in elements]
                homography_matrix.append(row)

        return np.array(homography_matrix).reshape(3, 3)

    def _read_mat_madk(self, file_path: str):
        mat_contents = scipy.io.loadmat(file_path)
        return mat_contents["grass"]

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        indices = self.indices[start:end]

        
        if self.double_input:
            X_images = []
            X_masks = []
        else:
            X = []
        y = []

        for i in indices:
            if self.double_input:
                image = preprocess_input(
                    cv2.resize(
                        cv2.cvtColor(
                            cv2.imread(self.input_paths[i][0]), cv2.COLOR_BGR2RGB
                        ),
                        self.image_size,
                    )
                )
                X_images.append(image)
                mask = self._read_mat_madk(self.input_paths[i][1])
                X_masks.append(mask)
            else:
                image = preprocess_input(
                    cv2.resize(
                        cv2.cvtColor(
                            cv2.imread(self.input_paths[i]), cv2.COLOR_BGR2RGB
                        ),
                        self.image_size,
                    )
                )
                X.append(image)

            matrix = self._read_homography(self.labels_list[i])
            y.append(matrix)

        if self.double_input:
            output = np.array(X_images), np.array(X_masks), np.array(y)
        else:
            output = np.array(X), np.array(y)
        
        return output

In [45]:
double_input = True

train_dataset = HomographyDataset(
        train_input,
        train_matrices,
        double_input=double_input,
        image_size=(540, 540),
        batch_size=1,
    )
test_dataset = HomographyDataset(
    val_input,
    val_matrices,
    double_input=double_input,
    image_size=(540, 540),
    batch_size=1,
)

In [48]:
train_dataset[0]

(array([[[[ 65,  62,  61],
          [ 54,  45,  46],
          [ 50,  39,  41],
          ...,
          [ 29,  44,  34],
          [ 32,  45,  36],
          [ 44,  57,  48]],
 
         [[ 60,  57,  57],
          [ 48,  38,  40],
          [ 46,  34,  37],
          ...,
          [ 39,  54,  44],
          [ 41,  56,  46],
          [ 48,  61,  52]],
 
         [[ 49,  45,  46],
          [ 39,  30,  33],
          [ 43,  31,  35],
          ...,
          [ 39,  55,  45],
          [ 44,  60,  50],
          [ 46,  59,  50]],
 
         ...,
 
         [[ 85, 140,  49],
          [ 85, 140,  49],
          [ 85, 140,  49],
          ...,
          [ 75, 123,  37],
          [ 75, 123,  37],
          [ 75, 123,  37]],
 
         [[ 85, 140,  49],
          [ 85, 140,  49],
          [ 85, 140,  49],
          ...,
          [ 75, 123,  37],
          [ 75, 123,  37],
          [ 75, 123,  37]],
 
         [[ 85, 140,  49],
          [ 85, 140,  49],
          [ 85, 140,  49],
   